In [ ]:
#twitter api
!pip install tweepy
import tweepy

In [ ]:
#this is needed to properly find the saved model in your google drive after mounting
#import os
#os.chdir('drive')
#os.chdir('MyDrive')

In [ ]:
#private twitter developer keys
apiKey = ""
apiKeySecret = ""
bearerToken = ""
accessToken = ""
accessTokenSecret = ""

In [ ]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(apiKey,apiKeySecret)
auth.set_access_token(accessToken, accessTokenSecret)

# Create API object
api = tweepy.API(auth)

# Create a tweet
#api.update_status("test")

In [ ]:
#@title ExtractData
#This function is taken from my N.B.A.A. project
#this function takes in the dataframes to return the appropriate features and tensors
def ExtractData(insertedData):
  
  data = insertedData

  #we insert tensors one row at a time into these list

  #features
  tempTensor = []
  #labels
  tempLabels = []

  #something important to note about this loop
  #two rows are 1 game, one for the away team one for the home team
  #somethings are updated every iteration, somethings every second
  for index, row in data.iterrows():

      #breaks up 'GSW @ NOP' into ['GSW','@','NOP']
      #matcup[0]&[2] hold the teams, [1] holds the orientation (home/away) for the team of that row
      matchup = row['MATCHUP'].split(" ")

      #updates the total Point Differential for a team at that point in the season
      totalPD[matchup[0]] += row['PLUS_MINUS']

      #home team
      if matchup[1] == "vs.":   
          
          #these basically take in a "fraction" in a string and update it
          #adjust win rate for home team win
          if row['PLUS_MINUS'] > 0:
            temp = homeTeamWLAtHome[matchup[0]]
            temp = temp.split("/")
            temp = str((int(temp[0])+1)) + "/" + str((int(temp[1])+1))         
            homeTeamWLAtHome[matchup[0]] = temp

          #adjust for loss
          elif row['PLUS_MINUS'] < 0:
            temp = homeTeamWLAtHome[matchup[0]]
            temp = temp.split("/")          
            temp = str((int(temp[0]))) + "/" + str((int(temp[1])+1))        
            homeTeamWLAtHome[matchup[0]] = temp
          
      #away team. here i do list operations
      #tracks results of last 10 away games
      elif matchup[1] == "@":
        if row['PLUS_MINUS'] > 0:
          
          #win. add 1
          #if there is less than 10 away games played we dont pop
          if len(awayTeamLastTen[teamDic[matchup[0]]]) > 9:
            awayTeamLastTen[teamDic[matchup[0]]].pop(0)
            awayTeamLastTen[teamDic[matchup[0]]].append(1)
          else:
            awayTeamLastTen[teamDic[matchup[0]]].append(1)

        #add zeros for losing a game
        elif row['PLUS_MINUS'] < 0:
          if len(awayTeamLastTen[teamDic[matchup[0]]]) > 9:
            awayTeamLastTen[teamDic[matchup[0]]].pop(0)
            awayTeamLastTen[teamDic[matchup[0]]].append(0)
          
          else:
            awayTeamLastTen[teamDic[matchup[0]]].append(0)
      
      #2rows is 1 game
      if index % 2 == 0:
        numberOfGamesTeamHasPlayed[matchup[0]] += 1
        numberOfGamesTeamHasPlayed[matchup[2]] += 1
      
      # format of the tensor features
      # |home team|away team| H.T win % @ H| A.T win % @ last 10 A| H.T avg PD | A.T avg PD
      else:

        homeTeam =""
        awayTeam =""
        didHomeTeamWin = -1

        #we use these if statements to figure out if the home team since if vs. its [0] and if its @ its [2]
        if matchup[1] == "@":
          #WL is respective to matchup 0
          awayTeam = matchup[0]
          homeTeam = matchup[2]
          
          if row['WL'] == 'L':
            didHomeTeamWin = 1
          else:
            didHomeTeamWin = 0
        else:
          awayTeam = matchup[2]
          homeTeam = matchup[0]

          if row['WL'] == 'W':
            didHomeTeamWin = 1
          else:
            didHomeTeamWin = 0


        homeTeamWinPercentAtHome = homeTeamWLAtHome[homeTeam].split("/")
        homeTeamWinPercentAtHome = float( int(homeTeamWinPercentAtHome[0])/int(homeTeamWinPercentAtHome[1])) 

        #this is to prevent 0/0 error
        if len(awayTeamLastTen[teamDic[awayTeam]]) == 0:
          awayTeamWinPercentForLastTen = 0.0
        else:
          awayTeamWinPercentForLastTen = float ( sum(awayTeamLastTen[teamDic[awayTeam]])/ len(awayTeamLastTen[teamDic[awayTeam]]) )
        
        
        homePD = float(totalPD[homeTeam]/numberOfGamesTeamHasPlayed[homeTeam])
        awayPD = float(totalPD[awayTeam]/numberOfGamesTeamHasPlayed[awayTeam])

        #add to the list
        tempTensor.append( tf.constant( [teamDic[homeTeam],teamDic[awayTeam],homeTeamWinPercentAtHome,awayTeamWinPercentForLastTen,homePD,awayPD] ) )
        tempLabels.append(tf.constant([didHomeTeamWin]))

  #this converts the list of 1D tensors into one tensor
  tempTensor = tf.stack(tempTensor)
  tempLabels = tf.stack(tempLabels)

  return tempTensor,tempLabels

In [ ]:
from google.colab import files
#load up current season data (currently using 2018-19 as practice data. Once 2021-22 season starts it will use whatever season data exist at that time)
uploaded = files.upload()

import numpy as np
import pandas as pd
import tensorflow as tf

teamDic = {"ATL":0,"BKN":1,"NJN":1,"BOS":2,"CHA":3,"CHI":4,"CLE":5,"DAL":6,"DEN":7,"DET":8,"GSW":9,"HOU":10,"IND":11,"LAC":12,"LAL":13,"MEM":14,"MIA":15,"MIL":16,"MIN":17,"NOP":18,"NOH":18,"NYK":19,"OKC":20,"ORL":21,"PHI":22,"PHX":23,"POR":24,"SAC":25,"SAS":26,"TOR":27,"UTA":27,"WAS":29}

homeTeamWLAtHome = dict.fromkeys(teamDic,"0/0")

awayTeamLastTen = [[] for i in range(30)]

numberOfGamesTeamHasPlayed = dict.fromkeys(teamDic, 0)
totalPD = dict.fromkeys(teamDic,0)

currentSeason = pd.read_csv("2018-19.csv") 

#we dont use these, we just need the data structures above to update
_, _ = ExtractData(currentSeason)


Saving 2018-19.csv to 2018-19.csv


In [ ]:
model = tf.keras.models.load_model('NBAA.v.0.1.h5')

probability_model = tf.keras.Sequential([model])
#predictions = probability_model.predict(testTensor)

In [ ]:
index = -1

#checks the last 20 mentions. If mention follows the proper format "@_NBAA_ MIA vs. BOS" and has not been liked by the bot,
#bot replies with the models prediction and then likes the tweet.
for tweet in api.mentions_timeline():

  index = index + 1
  #print(index,tweet.text,tweet.favorited,tweet.id_str, tweet.user.screen_name)
  if tweet.favorited == False:
    
    #splits prediction into a list that can be easily understood "@_NBAA_, MIA, vs., LAL,"
    predictionRequest = tweet.text.split(" ")

    #this makes sure the request is in the proper format
    if len(predictionRequest) == 4 and predictionRequest[2] == "vs.":
      
      homeTeamWinPercentAtHome = homeTeamWLAtHome[predictionRequest[1]].split("/")
      homeTeamWinPercentAtHome = float( int(homeTeamWinPercentAtHome[0])/int(homeTeamWinPercentAtHome[1])) 
      
      #this is to prevent 0/0 error
      if len(awayTeamLastTen[teamDic[predictionRequest[3]]]) == 0:
        awayTeamWinPercentForLastTen = 0.0
      else:
        awayTeamWinPercentForLastTen = float ( sum(awayTeamLastTen[teamDic[predictionRequest[3]]])/ len(awayTeamLastTen[teamDic[predictionRequest[3]]]) )

      homePD = float(totalPD[predictionRequest[1]]/numberOfGamesTeamHasPlayed[predictionRequest[1]])
      awayPD = float(totalPD[predictionRequest[3]]/numberOfGamesTeamHasPlayed[predictionRequest[3]])
     
      # |home team|away team| H.T win % @ H| A.T win % @ last 10 A| H.T avg PD | A.T avg PD
      inputTensor = tf.constant([ [ teamDic[predictionRequest[1]], teamDic[predictionRequest[3]], homeTeamWinPercentAtHome, awayTeamWinPercentForLastTen, homePD, awayPD  ] ])
      
      predictions = probability_model.predict(inputTensor)

      if predictions[0][0] < 0.5:
        status = "@"+tweet.user.screen_name+" "+predictionRequest[3]+ " wins"
      else:
        status = "@"+tweet.user.screen_name+" "+predictionRequest[1]+ " wins"

      api.update_status(status,in_reply_to_status_id = tweet.id_str)
      api.create_favorite(tweet.id_str)
    
    else:
      continue
        